In [0]:
%sql
-- UPDATE bronze.upsell.customers
-- SET QtdePontos
--  = 99999
-- WHERE IdCliente = "55dfeec5-2319-4aa5-af19-47e001bb0746";

In [0]:
%sql
DESCRIBE HISTORY bronze.upsell.customers;
SELECT * FROM table_changes('bronze.upsell.customers', 1, 2) WHERE IdCliente = 'd7f7eb75-4f0b-4d15-960d-8efe5ca7dd7e';

In [0]:
%sql
DESCRIBE HISTORY bronze.upsell.customers;

In [0]:
%sql
select * from table_changes('bronze.upsell.transactions', 1, 2)

In [0]:
%sql
drop table if exists bronze.upsell.products;
drop table if exists bronze.upsell.customers;
drop table if exists bronze.upsell.transactions;
drop table if exists bronze.upsell.transactions_product

In [0]:
spark.read\
  .format('parquet')\
  .load('/Volumes/raw/upsell/cdc/transactions/')\
  .createOrReplaceTempView('transactions_cdc')

In [0]:
%sql
select *
from transactions_cdc
qualify row_number() over (partition by IdTransacao order by change_timestamp desc) = 1 

In [0]:
df_transacoes = spark.read.csv('/Volumes/raw/full_load/full_load_csv/transacoes.csv')
df_transacoes.printSchema()
df_transacoes.show(10, truncate=False)

In [0]:
spark.read\
  .format('parquet')\
  .load('/Volumes/raw/upsell/full_load/transactions')\
  .createOrReplaceTempView('transactions_full_load_cdc')

In [0]:
%sql
select * from transactions_full_load_cdc

In [0]:
%sql
select *
from transactions_cdc
order by change_timestamp desc

In [0]:
spark.read\
  .format('parquet')\
  .load('/Volumes/raw/upsell/cdc/transactions_product/')\
  .createOrReplaceTempView('transactions_product_cdc')

In [0]:
%sql
select *
from transactions_product_cdc

In [0]:
spark.read\
      .format('parquet')\
      .load('/Volumes/raw/upsell/cdc/customers/')\
      .createOrReplaceTempView('customers_cdc')

In [0]:
%sql
select * 
from customers_cdc

In [0]:
%sql
select count(*) from bronze.upsell.customers

In [0]:
%sql
SELECT * FROM bronze.upsell.customers
        QUALIFY row_number() OVER (PARTITION BY IdCliente ORDER BY change_timestamp DESC) = 1

In [0]:
from pyspark.sql import SparkSession
from pathlib import Path

spark = SparkSession.builder.getOrCreate()

# Caminho do Parquet
parquet_path = "/Volumes/raw/upsell/full_load/products/"

# Ler o parquet
df = spark.read.parquet(parquet_path)

# Obter schema como JSON
schema_json = df.schema.json()

# Caminho de saída
output_path = Path("/Workspace/Users/robson.rogerio@gmail.com/lago-mago-tmw/src/bronze/products.json")

# Gravar arquivo
with open(output_path, "w") as f:
    f.write(schema_json)

print(f"✅ Schema salvo em: {output_path}")


In [0]:
%sql
select * from bronze.upsell.products